## Hash set membership demo

In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pytest"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import logging
FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
logging.basicConfig(format=FORMAT)
logging.getLogger().setLevel(logging.DEBUG)

# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

    def forward(self, x, y):
        diff = (x - y)
        membership_test = torch.prod(diff, dim=1)
        return (membership_test,y)


circuit = MyModel()

# Train the model as you like here (skipped for brevity)




DEBUG:pydot:pydot initializing
DEBUG:pydot:pydot 3.0.4
DEBUG:pydot.dot_parser:pydot dot_parser module initializing
DEBUG:pydot.core:pydot core module initializing


In [2]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [3]:
# print pytorch version
print(torch.__version__)

2.5.1+cu124


In [4]:


x = torch.zeros(1,*[1], requires_grad=True)
y = torch.tensor([0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0], requires_grad=True)

y_input = [0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

# Create an empty list to store the results
result = []

# Loop through each element in the y tensor
for e in y_input:
    # Apply the custom function and append the result to the list
    print(ezkl.float_to_felt(e,7))
    result.append(ezkl.poseidon_hash([ezkl.float_to_felt(e, 7)])[0])

y = y.unsqueeze(0)
y = y.reshape(1, 9)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      (x,y),                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=14,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array_x = ((x).detach().numpy()).reshape([-1]).tolist()
data_array_y = result
print(data_array_y)

data = dict(input_data = [data_array_x, data_array_y])

print(data)

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


0000000000000000000000000000000000000000000000000000000000000000
0001000000000000000000000000000000000000000000000000000000000000
8001000000000000000000000000000000000000000000000000000000000000
0002000000000000000000000000000000000000000000000000000000000000
8002000000000000000000000000000000000000000000000000000000000000
0003000000000000000000000000000000000000000000000000000000000000
8003000000000000000000000000000000000000000000000000000000000000
0004000000000000000000000000000000000000000000000000000000000000
8004000000000000000000000000000000000000000000000000000000000000
['c9ee727ef8f6307295e9e69831ecddb7866fa7c072dc626ad45a803258c2d411', '59a711784a1a751ed727c22a8ffb8f9f27171bd2d8f57fca2ba9ebf3dac3f023', '2ad51066722d0ce27aa8b16c8da8b229d009e8f804a49771309b3347f23ff504', '5f8c61fda484271ae4b14dd122a938ba74fa1cf3b1a4483ba4253b77c87a920a', 'a761d7c193856b47c39447bdd4a2a85fca8aebc604d8555299e01a5b5c39cc2c', '28c2facf747bda1cce78c265233ac09dda1708937c90af3b761ca579c248010d', 'e8670

In [5]:
run_args = ezkl.PyRunArgs()
# "hashed/private" means that the output of the hashing is not visible to the verifier and is instead fed into the computational graph
run_args.input_visibility = "hashed/private/0"
# as the inputs are felts we turn off input range checks
run_args.ignore_range_check_inputs_outputs = True
# we set it to fix the set we want to check membership for
run_args.param_visibility = "fixed"
# the output is public -- set membership fails if it is not = 0
run_args.output_visibility = "fixed"
run_args.variables = [("batch_size", 1)]
# never rebase the scale
run_args.scale_rebase_multiplier = 1000
# logrows
run_args.logrows = 11

#  this creates the following sequence of ops:
# 1. hash the input -> poseidon(x)
# 2. compute the set difference -> poseidon(x) - set
# 3. compute the product of the set difference -> prod(poseidon(x) - set)


# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True


DEBUG:tract_onnx.model:ONNX operator set version: 14
DEBUG:ezkl.graph.model:set batch_size to 1
DEBUG:tract_hir.infer.analyser:  Refined 2/0>: ..,? -> batch_size,9,F32
DEBUG:tract_hir.infer.analyser:  Refined 3/0>: ..,? -> batch_size,F32
DEBUG:tract_hir.infer.analyser:  Refined 4/0>: ..,? -> 1,9,F32
DEBUG:tract_core.optim:applying patch #0: declutter/0 >> declutter #5 "Identity_2" Identity
DEBUG:tract_core.optim.change_axes:  Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG:tract_core.optim.change_axes:    Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG:tract_core.optim.change_axes:  Considering change AxisChange { outlet: 0/0>, op: Rm(1) }
DEBUG:tract_core.optim.change_axes:    Change AxisChange { outlet: 0/0>, op: Rm(1) } blocked by locked interface 0/0>
DEBUG:tract_core.optim.change_axes:  Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG:tract_core.optim.change_axes:    Change AxisChange { outlet: 1/0>, op: Rm(0) } b

In [7]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

DEBUG:tract_onnx.model:ONNX operator set version: 14
DEBUG:ezkl.graph.model:set batch_size to 1
DEBUG:tract_hir.infer.analyser:  Refined 2/0>: ..,? -> batch_size,9,F32
DEBUG:tract_hir.infer.analyser:  Refined 3/0>: ..,? -> batch_size,F32
DEBUG:tract_hir.infer.analyser:  Refined 4/0>: ..,? -> 1,9,F32
DEBUG:tract_core.optim:applying patch #0: declutter/0 >> declutter #5 "Identity_2" Identity
DEBUG:tract_core.optim.change_axes:  Considering change AxisChange { outlet: 0/0>, op: Rm(0) }
DEBUG:tract_core.optim.change_axes:    Change AxisChange { outlet: 0/0>, op: Rm(0) } blocked by locked interface 0/0>
DEBUG:tract_core.optim.change_axes:  Considering change AxisChange { outlet: 0/0>, op: Rm(1) }
DEBUG:tract_core.optim.change_axes:    Change AxisChange { outlet: 0/0>, op: Rm(1) } blocked by locked interface 0/0>
DEBUG:tract_core.optim.change_axes:  Considering change AxisChange { outlet: 1/0>, op: Rm(0) }
DEBUG:tract_core.optim.change_axes:    Change AxisChange { outlet: 1/0>, op: Rm(0) } b

In [8]:
# srs path
res = await ezkl.get_srs( settings_path)

INFO:ezkl.execute:SRS does not exist, downloading...
DEBUG:reqwest.connect:starting new connection: https://trusted-setup-halo2kzg.s3.eu-central-1.amazonaws.com/
DEBUG:hyper_util.client.legacy.connect.dns:resolving host="trusted-setup-halo2kzg.s3.eu-central-1.amazonaws.com"
DEBUG:hyper_util.client.legacy.connect.http:connecting to 3.5.139.145:443
DEBUG:hyper_util.client.legacy.connect.http:connected to 3.5.139.145:443
DEBUG:hyper_util.client.legacy.pool:pooling idle connection for ("https", trusted-setup-halo2kzg.s3.eu-central-1.amazonaws.com)
INFO:ezkl.execute:Saving SRS to disk...
INFO:ezkl.execute:Saved SRS to /root/.ezkl/srs/kzg11.srs.
INFO:ezkl.execute:SRS downloaded
INFO:ezkl.execute:read 262404 bytes from file (vector of len = 262404)
INFO:ezkl.execute:file hash: 257fa566ed9bc0767d3e63e92b5e966829fa3347d320a32055dc31ee7d33f8a4


In [9]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

DEBUG:ezkl.graph:input scales: [7, 7]
DEBUG:ezkl.graph.model:calculating num of constraints using dummy model layout...
DEBUG:ezkl.graph.model:laying out 0: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=0, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.model:------------ output node 0: "Tensor { inner: [-149319993740704887712865435952675098936], dims: [1, 1], scale: None, visibility: None }"
DEBUG:ezkl.graph.model:------------ layout of 0 took 883.453µs
DEBUG:ezkl.graph.model:laying out 1: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=1, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.model:------------ output node 1: "Tensor { inner: [-149319993740704887712865435952675098936, 158656213775933422024196205498447538008, 198735708531470626446984873815

In [10]:
# now generate a faulty input + witness file (x input not in the set)

data_path_faulty = os.path.join('input_faulty.json')

witness_path_faulty = os.path.join('witness_faulty.json')

x = torch.ones(1,*[1], requires_grad=True)
y = torch.tensor([0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0], requires_grad=True)

y = y.unsqueeze(0)
y = y.reshape(1, 9)

data_array_x = ((x).detach().numpy()).reshape([-1]).tolist()
data_array_y = result
print(data_array_y)

data = dict(input_data = [data_array_x, data_array_y])

print(data)

    # Serialize data into file:
json.dump( data, open(data_path_faulty, 'w' ))

res = await ezkl.gen_witness(data_path_faulty, compiled_model_path, witness_path_faulty)
assert os.path.isfile(witness_path_faulty)

DEBUG:ezkl.graph:input scales: [7, 7]
DEBUG:ezkl.graph.model:calculating num of constraints using dummy model layout...
DEBUG:ezkl.graph.model:laying out 0: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=0, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.model:------------ output node 0: "Tensor { inner: [-65539363029252089940147629258685603718], dims: [1, 1], scale: None, visibility: None }"
DEBUG:ezkl.graph.model:------------ layout of 0 took 715.245µs
DEBUG:ezkl.graph.model:laying out 1: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=1, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.model:------------ output node 1: "Tensor { inner: [-149319993740704887712865435952675098936, 158656213775933422024196205498447538008, 1987357085314706264469848738151

['c9ee727ef8f6307295e9e69831ecddb7866fa7c072dc626ad45a803258c2d411', '59a711784a1a751ed727c22a8ffb8f9f27171bd2d8f57fca2ba9ebf3dac3f023', '2ad51066722d0ce27aa8b16c8da8b229d009e8f804a49771309b3347f23ff504', '5f8c61fda484271ae4b14dd122a938ba74fa1cf3b1a4483ba4253b77c87a920a', 'a761d7c193856b47c39447bdd4a2a85fca8aebc604d8555299e01a5b5c39cc2c', '28c2facf747bda1cce78c265233ac09dda1708937c90af3b761ca579c248010d', 'e86709c7126a9bf640640b5eb68833e2e11f20be881887716d567ef8310db70a', '7b5892a20a6dfc7b96f2ca5482608a9d349b120c571b1d338c2b807e1d955400', 'dbe3f2ef34b1f0dc9e98d3e5af291c1013511e70f935e30ff113f3df710ca42f']
{'input_data': [[1.0], ['c9ee727ef8f6307295e9e69831ecddb7866fa7c072dc626ad45a803258c2d411', '59a711784a1a751ed727c22a8ffb8f9f27171bd2d8f57fca2ba9ebf3dac3f023', '2ad51066722d0ce27aa8b16c8da8b229d009e8f804a49771309b3347f23ff504', '5f8c61fda484271ae4b14dd122a938ba74fa1cf3b1a4483ba4253b77c87a920a', 'a761d7c193856b47c39447bdd4a2a85fca8aebc604d8555299e01a5b5c39cc2c', '28c2facf747bda1cce78c2

In [11]:
# now generate a truthy input + witness file (x input not in the set)
import random

# Generate a random integer between 1 and 8, inclusive
random_value = random.randint(1, 8)

data_path_truthy = os.path.join('input_truthy.json')

witness_path_truthy = os.path.join('witness_truthy.json')

set = [0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

x = torch.tensor([set[random_value]])
y = torch.tensor(set, requires_grad=True)

y = y.unsqueeze(0)
y = y.reshape(1, 9)

x = x.unsqueeze(0)
x = x.reshape(1,1)

data_array_x = ((x).detach().numpy()).reshape([-1]).tolist()
data_array_y = result
print(data_array_y)

data = dict(input_data = [data_array_x, data_array_y])

print(data)

# Serialize data into file:
json.dump( data, open(data_path_truthy, 'w' ))

res = await ezkl.gen_witness(data_path_truthy, compiled_model_path, witness_path_truthy)
assert os.path.isfile(witness_path_truthy)

DEBUG:ezkl.graph:input scales: [7, 7]
DEBUG:ezkl.graph.model:calculating num of constraints using dummy model layout...
DEBUG:ezkl.graph.model:laying out 0: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=0, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.model:------------ output node 0: "Tensor { inner: [-32024760472861838207460578289777450022], dims: [1, 1], scale: None, visibility: None }"
DEBUG:ezkl.graph.model:------------ layout of 0 took 1.05943ms
DEBUG:ezkl.graph.model:laying out 1: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=1, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.model:------------ output node 1: "Tensor { inner: [-149319993740704887712865435952675098936, 158656213775933422024196205498447538008, 1987357085314706264469848738151

['c9ee727ef8f6307295e9e69831ecddb7866fa7c072dc626ad45a803258c2d411', '59a711784a1a751ed727c22a8ffb8f9f27171bd2d8f57fca2ba9ebf3dac3f023', '2ad51066722d0ce27aa8b16c8da8b229d009e8f804a49771309b3347f23ff504', '5f8c61fda484271ae4b14dd122a938ba74fa1cf3b1a4483ba4253b77c87a920a', 'a761d7c193856b47c39447bdd4a2a85fca8aebc604d8555299e01a5b5c39cc2c', '28c2facf747bda1cce78c265233ac09dda1708937c90af3b761ca579c248010d', 'e86709c7126a9bf640640b5eb68833e2e11f20be881887716d567ef8310db70a', '7b5892a20a6dfc7b96f2ca5482608a9d349b120c571b1d338c2b807e1d955400', 'dbe3f2ef34b1f0dc9e98d3e5af291c1013511e70f935e30ff113f3df710ca42f']
{'input_data': [[9.0], ['c9ee727ef8f6307295e9e69831ecddb7866fa7c072dc626ad45a803258c2d411', '59a711784a1a751ed727c22a8ffb8f9f27171bd2d8f57fca2ba9ebf3dac3f023', '2ad51066722d0ce27aa8b16c8da8b229d009e8f804a49771309b3347f23ff504', '5f8c61fda484271ae4b14dd122a938ba74fa1cf3b1a4483ba4253b77c87a920a', 'a761d7c193856b47c39447bdd4a2a85fca8aebc604d8555299e01a5b5c39cc2c', '28c2facf747bda1cce78c2

In [12]:
witness = json.load(open(witness_path, "r"))
witness

{'inputs': [['0000000000000000000000000000000000000000000000000000000000000000'],
  ['c9ee727ef8f6307295e9e69831ecddb7866fa7c072dc626ad45a803258c2d411',
   '59a711784a1a751ed727c22a8ffb8f9f27171bd2d8f57fca2ba9ebf3dac3f023',
   '2ad51066722d0ce27aa8b16c8da8b229d009e8f804a49771309b3347f23ff504',
   '5f8c61fda484271ae4b14dd122a938ba74fa1cf3b1a4483ba4253b77c87a920a',
   'a761d7c193856b47c39447bdd4a2a85fca8aebc604d8555299e01a5b5c39cc2c',
   '28c2facf747bda1cce78c265233ac09dda1708937c90af3b761ca579c248010d',
   'e86709c7126a9bf640640b5eb68833e2e11f20be881887716d567ef8310db70a',
   '7b5892a20a6dfc7b96f2ca5482608a9d349b120c571b1d338c2b807e1d955400',
   'dbe3f2ef34b1f0dc9e98d3e5af291c1013511e70f935e30ff113f3df710ca42f']],
 'pretty_elements': {'rescaled_inputs': [['0'],
   ['-1166562451099257000000000000000000000',
    '1239501670124479900000000000000000000',
    '15526227229021144000000000000000000',
    '-1142112490839209800000000000000000000',
    '575887733162474800000000000000000000',
    '

In [13]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK

# we force the output to be 0 this corresponds to the set membership test being true -- and we set this to a fixed vis output
# this means that the output is fixed and the verifier can see it but that if the input is not in the set the output will not be 0 and the verifier will reject
witness = json.load(open(witness_path, "r"))
witness["outputs"][0] = ["0000000000000000000000000000000000000000000000000000000000000000"]
json.dump(witness, open(witness_path, "w"))

witness = json.load(open(witness_path, "r"))
print(witness["outputs"][0])

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        witness_path = witness_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

DEBUG:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg11.srs"
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEBUG:ezkl.graph.vars:model uses 1 fixed columns
DEBUG:ezkl.graph.vars:model uses [] instance dims
DEBUG:ezkl.graph.model:configuring model
DEBUG:ezkl.graph:degree: 6, log2_ceil of degrees: 3.0
DEBUG:ezkl.graph:circuit size: 
 {
  "logrows": 11,
  "num_advice_columns": 9,
  "num_challenges": 0,
  "num_fixed": 5,
  "num_instances": 1,
  "num_selectors": 15
}
DEBUG:halo2_proofs.plonk.keygen:Creating domain with degree 6
DEBUG:ezkl.circuit.modules.planner:spawning module 0
DEBUG:ezkl.circuit.modules.planner:spawning module 2
INFO:ezkl.graph.model:model layout...
DEBUG:ezkl.graph.model:laying out 0: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=0, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.mod

['0000000000000000000000000000000000000000000000000000000000000000']


INFO:ezkl.pfsys:VK took 0.692
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEBUG:ezkl.graph.vars:model uses 1 fixed columns
DEBUG:ezkl.graph.vars:model uses [] instance dims
DEBUG:ezkl.graph.model:configuring model
DEBUG:ezkl.graph:degree: 6, log2_ceil of degrees: 3.0
DEBUG:ezkl.graph:circuit size: 
 {
  "logrows": 11,
  "num_advice_columns": 9,
  "num_challenges": 0,
  "num_fixed": 5,
  "num_instances": 1,
  "num_selectors": 15
}
DEBUG:ezkl.circuit.modules.planner:spawning module 0
DEBUG:ezkl.circuit.modules.planner:spawning module 2
INFO:ezkl.graph.model:model layout...
DEBUG:ezkl.graph.model:laying out 0: Input
DEBUG:ezkl.circuit.ops.region:(rows=0, coord=0, constants=0, max_lookup_inputs=0, min_lookup_inputs=0, max_range_size=0, dynamic_lookup_col_coord=0, shuffle_col_coord=0, max_dynamic_input_len=0)
DEBUG:ezkl.graph.model:------------ output node 0: "Tensor { inner: [], dims: [0], scale: None, visibility: None }"
DE

In [14]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
assert os.path.isfile(proof_path)

DEBUG:ezkl.graph:rescaled and processed public inputs: {
  "inputs": [],
  "outputs": [],
  "processed_inputs": [
    [
      "0x11d4c25832805ad46a62dc72c0a76f86b7ddec3198e6e9957230f6f87e72eec9"
    ]
  ],
  "processed_outputs": [],
  "processed_params": [],
  "rescaled_inputs": [],
  "rescaled_outputs": []
}
DEBUG:ezkl.graph:public inputs: [0x11d4c25832805ad46a62dc72c0a76f86b7ddec3198e6e9957230f6f87e72eec9]
DEBUG:ezkl.pfsys:loading proving key from "test.pk"
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEBUG:ezkl.graph.vars:model uses 1 fixed columns
DEBUG:ezkl.graph.vars:model uses [] instance dims
DEBUG:ezkl.graph.model:configuring model
DEBUG:ezkl.graph:degree: 6, log2_ceil of degrees: 3.0
DEBUG:ezkl.graph:circuit size: 
 {
  "logrows": 11,
  "num_advice_columns": 9,
  "num_challenges": 0,
  "num_fixed": 5,
  "num_instances": 1,
  "num_selectors": 15
}
DEBUG:halo2_proofs.plonk.keygen:Creating domain with degree 6
INFO

{'instances': [['c9ee727ef8f6307295e9e69831ecddb7866fa7c072dc626ad45a803258c2d411']], 'proof': '0x28576823741f9084010b6b5d18bfaee7520e3944e6130af7c5e05f5d15a9cb4221e69aefb133eb9764a6f844ca8d21c8f4d6fe62bd0397123f015da87f258e580ade8c007de32460bd2001d353ed5141628d6b43d41e05d5c6178f4cd53e80c6273f7b7a85ce4055724bcf9eff29c0c74edaa20abeee975a694005335c09c7d00426011d37d5af7ef9ea9ab45b3b8f333a9fe336730cfa9cca53398d1082e6f2185757c1cb580f44f68b1886ebbb9db112d26112d83684632197e90411c132e81241b43c4519d15474bd49e6df24249d62d92e31593320e7fb7b6560b73134ce065e6563ad16a3f89ecb1692620d86e9257d84de86a25747c53bc58186b4a6c6005004a24d30b12bd557f0a51677460881edc7903c7510166b493e51d2ac882407649664b558e5bf39c1649ef018684ec5a0aea989213997809691508ea39e4917a945ff976ced38b063ad0552c805829be1feb5053cd9ed952a72be41a265fe0482c8f9c75a23884bc7a0476dcde9c78a469e27f00c4892222ce10c8df3aa0e0b3233bcfce054bb783a72545b6fcafa16d7794fc26a37aff1ab3d890c00efb52b3b9c8dc813da6b466e629c4f3847ffc07439cffafb441bdbacd709fc3e4eea063dbd

In [15]:
# GENERATE A FAULTY PROOF


proof_path_faulty = os.path.join('test_faulty.pf')

res = ezkl.prove(
        witness_path_faulty,
        compiled_model_path,
        pk_path,
        proof_path_faulty,

        "single",
    )

print(res)
assert os.path.isfile(proof_path_faulty)

DEBUG:ezkl.graph:rescaled and processed public inputs: {
  "inputs": [],
  "outputs": [],
  "processed_inputs": [
    [
      "0x25f4ed3b53150e4a0d7cb81fbf7a52b9f6e57c34869170c5324d870010c7207b"
    ]
  ],
  "processed_outputs": [],
  "processed_params": [],
  "rescaled_inputs": [],
  "rescaled_outputs": []
}
DEBUG:ezkl.graph:public inputs: [0x25f4ed3b53150e4a0d7cb81fbf7a52b9f6e57c34869170c5324d870010c7207b]
DEBUG:ezkl.pfsys:loading proving key from "test.pk"
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEBUG:ezkl.graph.vars:model uses 1 fixed columns
DEBUG:ezkl.graph.vars:model uses [] instance dims
DEBUG:ezkl.graph.model:configuring model
DEBUG:ezkl.graph:degree: 6, log2_ceil of degrees: 3.0
DEBUG:ezkl.graph:circuit size: 
 {
  "logrows": 11,
  "num_advice_columns": 9,
  "num_challenges": 0,
  "num_fixed": 5,
  "num_instances": 1,
  "num_selectors": 15
}
DEBUG:halo2_proofs.plonk.keygen:Creating domain with degree 6
INFO

{'instances': [['7b20c71000874d32c5709186347ce5f6b9527abf1fb87c0d4a0e15533bedf425']], 'proof': '0x22028ffd1b0ed56e84ba0eaf4e3afbfdb98ef9990ec8eff98f6fde32b42ddc86188928f526adaa365ad458a98c3d31499681224fc1785b98cd13f61b21cf0773103638de28cb89264d6fb5b60df19b41ba3242b7969a3088fe9121ab424e39bf14d85710923acd58d390339a56ac96ab27d79b77c7716a10d1c6d474b40d69e22d45f9b48142ef2fc50b651bc30c9a2fcf9c482bebd1c252df06a310f7f5ff3611c6c87f1dfc77bc03cf6633f5a96f4a677001a0dad5db2172eb2691ae88eb672c1b9bc42eb161c95dc845fa75f30c9728250cd863eb8a6ecbb536174b6abca61accfca8a39c985b22092d5a0d2ae4d0267f2ba6d765f8fda1d313731e1ebe7311a7f6aa2d5650cb31fd6a8795c84fabc927d81638c02e4456fc961525a9b556043c5237eab006ccb749bfb4804d90bf52b3d247a44ad81984779d188c5c132e09251659582e9f4608800d4868e71e1e27445f8a9b3aa3830f188d7b7b4fc4d51f08fb54874159ba3d778474228acfc4da9d863a38bb76099c85254b7b4e79ea2c4e78d2ccd920e4e9476aeae90ef8dab7e2b7b5aa986bb169cf493b1747643f0effa93e56954a20fcfbecd03c5c7ca9b0f55ac22d94843d3d98fc79658d234d284836

In [16]:
# GENERATE A TRUTHY PROOF


proof_path_truthy = os.path.join('test_truthy.pf')

res = ezkl.prove(
        witness_path_truthy,
        compiled_model_path,
        pk_path,
        proof_path_truthy,

        "single",
    )

print(res)
assert os.path.isfile(proof_path_truthy)

DEBUG:ezkl.graph:rescaled and processed public inputs: {
  "inputs": [],
  "outputs": [],
  "processed_inputs": [
    [
      "0x2fa40c71dff313f10fe335f9701e5113101c29afe5d3989edcf0b134eff2e3db"
    ]
  ],
  "processed_outputs": [],
  "processed_params": [],
  "rescaled_inputs": [],
  "rescaled_outputs": []
}
DEBUG:ezkl.graph:public inputs: [0x2fa40c71dff313f10fe335f9701e5113101c29afe5d3989edcf0b134eff2e3db]
DEBUG:ezkl.pfsys:loading proving key from "test.pk"
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEBUG:ezkl.graph.vars:model uses 1 fixed columns
DEBUG:ezkl.graph.vars:model uses [] instance dims
DEBUG:ezkl.graph.model:configuring model
DEBUG:ezkl.graph:degree: 6, log2_ceil of degrees: 3.0
DEBUG:ezkl.graph:circuit size: 
 {
  "logrows": 11,
  "num_advice_columns": 9,
  "num_challenges": 0,
  "num_fixed": 5,
  "num_instances": 1,
  "num_selectors": 15
}
DEBUG:halo2_proofs.plonk.keygen:Creating domain with degree 6
INFO

{'instances': [['dbe3f2ef34b1f0dc9e98d3e5af291c1013511e70f935e30ff113f3df710ca42f']], 'proof': '0x1653c09d7380a5f5df6bcb3cdce8b5795c74a0548db859ecd81ea27216731ff00bb4d79fca1f8012b0f58831aa29f672b2e88ed159d8df9430236eee42f01778099872ecba723b3136796d59e1d5a681eae077bef560ec8bff9c6d88be83f0742f372fd114558e08067470b0b8057f9b2f109f35c2352049f61b1994f1189cf1210f516b18a7f22605f410bd53be8447d72e234774e6d2d19e235f6e9a4fb03f13ab0cbcfc7172b08395c4bb1fc15e2afc6f3de410e4b465959a565218f0cc172be6f1ae18dc21dcbc9208c4945440f52bc40fd09f5d2eedd2962e937d6d534523136e5ff510ca864f2e0615accc7f52240d4a664f596edce321b493f5810f680169bba1784e0daa0878913d41c9e2be9820965f4ba1b9532ce8ef7f046f0940075fec64b4a116a5429598ac65d2f8bba006a3568194283769248214f53e1fd229e1c79beec42b6f2a5090a512148b8667a581564f17f6a1afdf04b67889dfce002c70c2a17e36229061caaff26482a85f46ce24d9d665bd4abedd1826124d66246dcfd6f7745412c4b3f7bbc8ff5e08bcc2295fe9b057f730c0bf2b64ce66cc214bbb8002a0c79f90397298c35b2d9d96817ae9903ebd52a5ca461e559d2d1d217da5

In [17]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,

    )
assert res == True

res = ezkl.verify(
        proof_path_truthy,
        settings_path,
        vk_path,

    )
assert res == True

DEBUG:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg11.srs"
DEBUG:ezkl.pfsys:loading verification key from "test.vk"
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEBUG:ezkl.graph.vars:model uses 1 fixed columns
DEBUG:ezkl.graph.vars:model uses [] instance dims
DEBUG:ezkl.graph.model:configuring model
DEBUG:ezkl.graph:degree: 6, log2_ceil of degrees: 3.0
DEBUG:ezkl.graph:circuit size: 
 {
  "logrows": 11,
  "num_advice_columns": 9,
  "num_challenges": 0,
  "num_fixed": 5,
  "num_instances": 1,
  "num_selectors": 15
}
DEBUG:halo2_proofs.plonk.keygen:Creating domain with degree 6
INFO:ezkl.pfsys:loaded verification key ✅
INFO:ezkl.execute:verify took 0.6
INFO:ezkl.execute:verified: true
DEBUG:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg11.srs"
DEBUG:ezkl.pfsys:loading verification key from "test.vk"
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEB

In [18]:
import pytest
def test_verification():
    with pytest.raises(RuntimeError, match='Failed to run verify: \\[halo2\\] The constraint system is not satisfied'):
        ezkl.verify(
            proof_path_faulty,
            settings_path,
            vk_path,

        )

# Run the test function
test_verification()

DEBUG:ezkl.pfsys.srs:loading srs from "/root/.ezkl/srs/kzg11.srs"
DEBUG:ezkl.pfsys:loading verification key from "test.vk"
DEBUG:ezkl.graph.vars:number of blinding factors: 5
DEBUG:ezkl.graph.vars:model uses 3 advice blocks (size=2)
DEBUG:ezkl.graph.vars:model uses 1 fixed columns
DEBUG:ezkl.graph.vars:model uses [] instance dims
DEBUG:ezkl.graph.model:configuring model
DEBUG:ezkl.graph:degree: 6, log2_ceil of degrees: 3.0
DEBUG:ezkl.graph:circuit size: 
 {
  "logrows": 11,
  "num_advice_columns": 9,
  "num_challenges": 0,
  "num_fixed": 5,
  "num_instances": 1,
  "num_selectors": 15
}
DEBUG:halo2_proofs.plonk.keygen:Creating domain with degree 6
INFO:ezkl.pfsys:loaded verification key ✅
INFO:ezkl.execute:verify took 0.8
INFO:ezkl.execute:verified: false
